#### 1. Install Dependencies

In [ ]:
pip install -r requirements.txt -q

In [ ]:
pip show pinecone-client

#### 2. Verify Pinecone

In [ ]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

pinecone.init(
  api_key=os.environ.get("PINECONE_API_KEY"),
  environment=os.environ.get("PINECONE_ENV")
)
pinecone.info.version()

#### 3. Pinecone Indexes

In [ ]:
idx_name = 'langchain-pinecone'

####################
### create index ###
####################

# only one index allowed in free account
if idx_name not in pinecone.list_indexes():
  pinecone.create_index(name=idx_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
else:
  print(f'Index {idx_name} already exists. Not creating one.')
  
######################
### describe index ###
######################

pinecone.describe_index(idx_name)
pinecone.Index(idx_name).describe_index_stats()

#### 6. Pinecone Vectors

In [25]:
import random
idx_name = 'langchain-pinecone'

######################
### create vectors ###
######################

# create 10 random vectors
# each vector is a list of 1536 floats
vectors = [[random.random() for _ in range(1536)] for _ in range(5)]

# unqiue id for each vector from above
ids = list('ABCDE')
idx = pinecone.Index(idx_name)
idx.upsert(vectors=zip(ids, vectors))

######################
### update vectors ###
######################

# update vectors with id 'A'
updated_vectors = [random.random() for _ in range(1536)]
idx = pinecone.Index(idx_name)
idx.upsert(vectors=[('A', updated_vectors)])

#####################
### fetch vectors ###
#####################

# fetch vectors with id 'A' and 'C'
idx = pinecone.Index(idx_name)
idx.fetch(ids=['A', 'C'])

#####################
### query vectors ###
#####################

# create two random vectors to search for
random_vector = [[random.random() for _ in range(1536)] for _ in range(2)]
idx = pinecone.Index(idx_name)
idx.query(
  vector=random_vector, 
  top_k=2,
  include_values=False
)

{'matches': [{'id': 'E', 'score': 0.749005854, 'values': []},
             {'id': 'D', 'score': 0.748185635, 'values': []}],
 'namespace': ''}

#### 8. Delete Vectors & Indexes

In [23]:
############################
### delete vectors by id ###
############################

# delete vectors with id 'B' and 'D'
# idx = pinecone.Index(idx_name)
# idx.delete(ids=['B', 'D'])

##########################
### delete all vectors ###
##########################

# idx = pinecone.Index(idx_name)  
# idx.delete_all()

####################
### delete index ###
####################

# if idx_name in pinecone.list_indexes():
#   pinecone.delete_index(idx_name)
# else:
#   print(f'Index {idx_name} does not exist.')

{'matches': [{'id': 'D', 'score': 0.770377219, 'values': []},
             {'id': 'A', 'score': 0.75805676, 'values': []},
             {'id': 'E', 'score': 0.757711649, 'values': []}],
 'namespace': ''}